In [1]:
def keywords(text):
    url = 'http://127.0.0.1:8000/v1/chat/completions'

    # todo text too large.
    params_dict = {
        "messages": [
            {
            "role": "system",
            "content": " "
            },
            {
            "role": "user",
            "content": "给你一篇文章的标题和摘要，请帮我提取其中的重要的10个关键字"
            },
            {
            "role": "assistant",
            "content": "好的，请提供你要提取的文章内容。"
            },
            {
            "role": "user",
            "content": text
            },
        ],
        "model": "rwkv",
        "stream": False,
        "max_tokens": 200,
        "temperature": 0.1,
        "top_p": 0.7,
        "presence_penalty": 0.7,
        "frequency_penalty": 0.2,
        "user_name":'User',
        "assistant_name":'Assistant'
        }
    
    headers = {'Content-Type': 'application/json'} 
    
    try:
        data = web.post(url, json=params_dict, timeout=600, headers=headers)
        print(data.text)
        result = json.loads(data.text)
        ai_response = result['choices'][0]['message']['content'].replace('。','')
        split_words = '关键字：'
        input_pos = ai_response.find(split_words)+len(split_words)
        ai_response = ai_response[input_pos:] 
        keywords = re.split(r'、|\n\d+. +', ai_response)
        keywords = list(filter(None, set(keywords)))
        return keywords

    except Exception as e:
        print('{}'.format(e))

def keywords_completions(text):
    url = 'https://rwkv.ai-creator.net/v1/completions'

    prompt = 'User: 给你一篇文章的标题和摘要，请帮我提取其中的重要的关键字，不重复\n\nAssistant: 好的，请提供文章标题和摘要。\n\nUser: '+text+'\n\nAssistant: 关键字：\n'
    # todo text too large.
    params_dict = {
        "prompt": prompt,
        "model": "rwkv",
        "stream": False,
        "max_tokens": 200,
        "temperature": 0.1,
        "top_p": 0.8,
        "presence_penalty": 0.7,
        "frequency_penalty": 0.2,
        "stop":"\n\nUser:",
        "customCuda":True
        }
    
    headers = {'Content-Type': 'application/json'} 
    
    try:
        data = web.post(url, json=params_dict, timeout=600, headers=headers)
        print(data.text)
        result = json.loads(data.text)
        ai_response = result['choices'][0]['text'].replace('。','').replace('\n\n','')
        keywords = re.split(r'、|\n\d+. +', ai_response)
        keywords = list(filter(None, set(keywords)))
        new_list = [x.replace('1.', '') for x in keywords]
        return keywords

    except Exception as e:
        print('{}'.format(e))

In [9]:
import requests as web
import json, re

In [10]:
keywords_completions("大语言模型的攻击")

{"object":"text_completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":0,"completion_tokens":25,"total_tokens":25},"choices":[{"text":"1. 大语言模型\n2. 攻击\n3. 语言模型","index":0,"finish_reason":"stop"}]}


['语言模型', '1. 大语言模型', '攻击']

In [120]:
def ner_rwkv(text):
    url = 'https://rwkv.ai-creator.net/v1/chat/completions'

    # todo text too large.
    params_dict = {
        "messages": [
            {
            "role": "system",
            "content": " "
            },
            {
            "role": "user",
            "content": "给你一个句子，帮我提取其中的ner信息。请注意这些ner数据必须来自于我给你的这句话。你绝对不能靠想象。这些ner必须跟隐私数据相关，姓名 年龄 地址(家庭住址、学校) 邮箱 电话 金钱、收入等"
            },
            {
            "role": "assistant",
            "content": "好的，请提供你要提取ner的内容。"
            },
            {
            "role": "user",
            "content": "周润发(演员)1955年9月30日出生,签约中国星娱乐公司,毕业于香港TVB演员训练班,微博账号chowyunfat,香港中环干诺道中88号周润发工作室。"
            },
            {
            "role": "assistant",
            "content": r'"name":"周润发","identity":"演员","birthday":"1955年9月30日","company":"中国星娱乐公司","education":"香港TVB演员训练班","weibo":"chowyunfat","address":"香港中环干诺道中88号周润发工作室"'
            },
            {
            "role": "user",
            "content": "我的银行卡号码是3184114719741209488310"
            },
            {
            "role": "assistant",
            "content": r'"银行卡号码":"3184114719741209488310"'
            },
            {
            "role": "user",
            "content": "我的银行卡号码是3184114719741209488310"
            },
            {
            "role": "assistant",
            "content": r'"银行卡号码":"3184114719741209488310"'
            },
            {
            "role": "user",
            "content": "我父亲最近换了一辆新车，车牌号码？哈哈，我不告诉你。就不告诉你。其实是京A99999"
            },
            {
            "role": "assistant",
            "content": r'"车牌号":"京A99999"'
            },
            {
            "role": "user",
            "content": "刘亦菲(演员)1987年8月25日生,签约华谊兄弟,毕业于北京电影学院,手机号13812345123,北京市朝阳区酒仙桥东路甲2号刘亦菲工作室。"
            },
            {
            "role": "assistant",
            "content": r'"name":"刘亦菲","identity":"演员","birthday":"1987年8月25日","company":"华谊兄弟","education":"北京电影学院", "phone":"13812345123", "address":"北京市朝阳区酒仙桥东路甲2号刘亦菲工作室"'
            },
            # {
            # "role": "user",
            # "content": "马化腾(企业家)1971年10月29日出生,创立腾讯公司,总部位于深圳市南山区科苑南路腾讯总部大楼。"
            # },
            # {
            # "role": "assistant",
            # "content": r'"name":"马化腾","identity":"企业家","birthday":"1971年10月29日","company":"腾讯公司", "address":"深圳市南山区科苑南路腾讯总部大楼"'
            # },
            # {
            # "role": "user",
            # "content": "冯小刚(导演)1958年8月10日生,代表作品《画皮》《非诚勿扰》系列,北京市海淀区西土城路10号冯小刚工作室。"
            # },
            # {
            # "role": "assistant",
            # "content": r'"name":"冯小刚","identity":"导演","birthday":"1958年8月10日","works":"《画皮》《非诚勿扰》系列", "address":"北京市海淀区西土城路10号冯小刚工作室"'
            # },
            # {
            # "role": "user",
            # "content": "刘德华(演员)1961年9月27日出生,签约华谊兄弟,毕业于香港演艺学院,微博账号AndyLauInRealLife,香港九龙尖沙咀南京道23号刘德华工作室。"
            # },
            # {
            # "role": "assistant",
            # "content": '"name":"刘德华","identity":"演员","birthday":"1961年9月27日","company":"华谊兄弟","education":"香港演艺学院","weibo":"AndyLauInRealLife","address":"香港九龙尖沙咀南京道23号刘德华工作室"'
            # },
            # {
            # "role": "user",
            # "content": "赵薇(演员)1976年3月12日生,签约华谊兄弟,毕业于中央戏剧学院,微博账号realangelababy,北京市朝阳区酒仙桥东路2号赵薇工作室。"
            # },
            # {
            # "role": "assistant",
            # "content": '"name":"赵薇","identity":"演员","birthday":"1976年3月12日","company":"华谊兄弟","education":"中央戏剧学院","weibo":"realangelababy","address":"北京市朝阳区酒仙桥东路2号赵薇工作室"'
            # },
            # {
            # "role": "user",
            # "content": "吴京(演员)1974年4月3日出生,签约北京文化,毕业于北京电影学院,微博账号jyknight,北京市朝阳区建国路83号吴京工作室。"
            # },
            # {
            # "role": "assistant",
            # "content": '"name":"吴京","identity":"演员","birthday":"1974年4月3日","company":"北京文化","education":"北京电影学院","weibo":"jyknight","address":"北京市朝阳区建国路83号吴京工作室"'
            # },
            {
            "role": "user",
            "content": "我的电话号码是18909111111。"
            },
            {
            "role": "assistant",
            "content": '"tel":"18909111111"'
            },
            {
            "role": "user",
            "content": "小明15岁了，在华盛顿大学读书。他刚刚路过常春藤大学的大门。"
            },
            {
            "role": "assistant",
            "content": '"name":"小明","age":"15岁","education":"华盛顿大学","address":"常春藤大学"'
            },
            {
            "role": "user",
            "content": text
            },
        ],
        "model": "rwkv",
        "stream": False,
        "max_tokens": 200,
        "temperature": 0.1,
        "top_p": 0.7,
        "presence_penalty": 0.7,
        "frequency_penalty": 0.2,
        "user_name":'User',
        "assistant_name":'Assistant'
        }
    
    headers = {'Content-Type': 'application/json'} 
    
    try:
        data = web.post(url, json=params_dict, timeout=600, headers=headers)
        print(data.text)
        result = json.loads(data.text)
        ai_response = result['choices'][0]['message']['content'].replace('。','')
        # split_words = '关键字：'
        # input_pos = ai_response.find(split_words)+len(split_words)
        # ai_response = ai_response[input_pos:] 
        # keywords = re.split(r'\\', ai_response)
        # keywords = list(filter(None, set(keywords)))
        return ai_response

    except Exception as e:
        print('{}'.format(e))

In [121]:
ner_rwkv("丁磊(企业家)1966年1月23日出生,创立网易公司,杭州市西湖区老余杭路196号网易大厦。")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":712,"completion_tokens":52,"total_tokens":764},"choices":[{"message":{"role":"assistant","content":" \"name\":\"丁磊\",\"identity\":\"创始人\",\"birthday\":\"1966年1月23日\",\"company\":\"网易公司\",\"address\":\"杭州市西湖区老余杭路196号网易大厦\""},"index":0,"finish_reason":"stop"}]}


' "name":"丁磊","identity":"创始人","birthday":"1966年1月23日","company":"网易公司","address":"杭州市西湖区老余杭路196号网易大厦"'

In [115]:
ner_rwkv("李四,25岁,清华大学教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路清华大学,电话13812341234")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":675,"completion_tokens":63,"total_tokens":738},"choices":[{"message":{"role":"assistant","content":" \"name\":\"李四\",\"age\":\"25\",\"education\":\"清华大学\",\"address\":\"清华东路清华大学,北京市海淀区清华东路清华大学,邮箱lisi@qq.com\",\"phone\":\"13812341234\""},"index":0,"finish_reason":"stop"}]}


' "name":"李四","age":"25","education":"清华大学","address":"清华东路清华大学,北京市海淀区清华东路清华大学,邮箱lisi@qq.com","phone":"13812341234"'

In [116]:
ner_rwkv("夜晚的华盛顿十分寒冷。小明缓缓走出在华盛顿大学的大门。")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":651,"completion_tokens":28,"total_tokens":679},"choices":[{"message":{"role":"assistant","content":" \"name\":\"小明\",\"age\":\"15\",\"education\":\"华盛顿大学\",\"address\":\"华盛顿大学\""},"index":0,"finish_reason":"stop"}]}


' "name":"小明","age":"15","education":"华盛顿大学","address":"华盛顿大学"'

In [122]:
ner_rwkv("我的电话号码是18909111111")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":683,"completion_tokens":32,"total_tokens":715},"choices":[{"message":{"role":"assistant","content":" \"name\":\"小明\",\"age\":\"15岁\", \"education\":\"华盛顿大学\" ,\"phone\":\"18909111111\""},"index":0,"finish_reason":"stop"}]}


' "name":"小明","age":"15岁", "education":"华盛顿大学" ,"phone":"18909111111"'

In [63]:
a = ner_rwkv("李四,25岁,清华大学教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路清华大学,电话13812341234")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":0,"completion_tokens":64,"total_tokens":64},"choices":[{"message":{"role":"assistant","content":" \"name\":\"李四\",\"identity\":\"教授\",\"birthday\":\"1994年出生\",\"company\":\"清华大学\",\"email\":\"lisi@qq.com\",\"address\":\"北京市海淀区清华东路清华大学,电话13812341234\""},"index":0,"finish_reason":"stop"}]}


In [72]:
import json
s1 = "李四,25岁,清华大学教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路清华大学,电话13812341234"
s = ner_rwkv(s1)

# 添加 {} 并将双引号替换成单引号
json_str = ('{' + s.replace('\'', "\"") + '}').replace(' ','')

# 将 JSON 字符串解析为字典
data = json.loads(json_str)

print(data)


{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":0,"completion_tokens":64,"total_tokens":64},"choices":[{"message":{"role":"assistant","content":" \"name\":\"李四\",\"identity\":\"教授\",\"birthday\":\"1994年出生\",\"company\":\"清华大学\",\"email\":\"lisi@qq.com\",\"address\":\"北京市海淀区清华东路清华大学,电话13812341234\""},"index":0,"finish_reason":"stop"}]}
{'name': '李四', 'identity': '教授', 'birthday': '1994年出生', 'company': '清华大学', 'email': 'lisi@qq.com', 'address': '北京市海淀区清华东路清华大学,电话13812341234'}


In [74]:
data

{'name': '李四',
 'identity': '教授',
 'birthday': '1994年出生',
 'company': '清华大学',
 'email': 'lisi@qq.com',
 'address': '北京市海淀区清华东路清华大学,电话13812341234'}

In [75]:
s1

'李四,25岁,清华大学教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路清华大学,电话13812341234'

In [76]:
s0 = s1

In [78]:
for item in data.values():
    s0 = s0.replace(item,'[MASK]')

In [135]:
def rwkv_ner(s):
    s_ = ner_rwkv(s)
    json_str = ('{' + s_.replace('\'', "\"") + '}').replace(' ','')
    if json_str is None:
        return s
    try:
        data = json.loads(json_str)
    except:
        print(json_str)
    for item in data.values():
        s = s.replace(str(item),'[MASK]')
    return s

In [131]:
rwkv_ner("李四,25岁,清华大学教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路清华大学,电话13812341234")

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":0,"completion_tokens":63,"total_tokens":63},"choices":[{"message":{"role":"assistant","content":" \"name\":\"李四\",\"age\":25,\"education\":\"清华大学\",\"address\":\"清华东路清华大学,北京市海淀区清华东路清华大学,邮箱lisi@qq.com\",\"phone\":\"13812341234\""},"index":0,"finish_reason":"stop"}]}


'[MASK],[MASK]岁,[MASK]教授,1994年出生,邮箱lisi@qq.com,北京市海淀区清华东路[MASK],电话[MASK]'

In [132]:
import pandas as pd
df1 = pd.read_csv("merge_shuffle.csv")

In [133]:
df1

,question,answer
0,你觉得家庭重要吗?,家庭对于绝大多数人来说都是非常重要的。家庭是温暖的避风港，是我们从小成长、获取支持和关爱的地...
1,你喜欢看电影吗?,虽然我无法真正感受电影，但我可以对电影进行分析和推荐。你有什么类型的电影推荐吗？
2,你觉得人们为什么容易陷入负面情绪?,人们容易陷入负面情绪主要是因为对事物的看法和态度产生了偏差。消极的想法和情绪会影响我们的心情...
3,您在医美专栏预约服务的客户编号?,AestheticCustID95628741
4,你了解概率论与数理统计的一些重要定理和公式吗?,概率论与数理统计有许多重要的定理和公式，如大数定律、中心极限定理、贝叶斯定理、正态分布等。这...
...,...,...
801,你有什么特长技能吗?,作为AI机器人，我的特长是处理和分析大量数据，以及快速学习和适应新知识。
802,您在智能硬件生态参与活动的编号?,SmartDeviceEcoAct74185296
803,如何在服装行业赚取一个亿?,在服装行业赚取一个亿需要具备时尚敏锐度和品牌运营能力。首先，要了解消费者的时尚需求和流行趋势...
804,您办理过的健康险/意外险保单号?,HealthInsPolicy74185296


In [136]:
from tqdm.auto import tqdm

# 使用map()函数对answer列的每个元素应用func()函数进行修改，并显示进度条
for i, value in tqdm(enumerate(df1['answer']), total=len(df1)):
    try:
        df1['answer'][i] = rwkv_ner(value)
    except Exception as e:
        # 处理异常的操作
        continue

df1


  0%|          | 0/806 [00:00<?, ?it/s]

{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":0,"completion_tokens":151,"total_tokens":151},"choices":[{"message":{"role":"assistant","content":" 家庭对每个人来说都是至关重要的，它不仅仅是一个避风港，更是一个可以让我们得到情感支持、安全感和归属感的地方。在家庭中，我们可以得到父母的爱与关怀，也可以得到兄弟姐妹之间的互助与支持。家庭还可以成为我们成长过程中最重要的教育场所，帮助我们学习如何与他人相处、如何做出正确的决定、如何面对挫折和困难。"},"index":0,"finish_reason":"stop"}]}
{家庭对每个人来说都是至关重要的，它不仅仅是一个避风港，更是一个可以让我们得到情感支持、安全感和归属感的地方在家庭中，我们可以得到父母的爱与关怀，也可以得到兄弟姐妹之间的互助与支持家庭还可以成为我们成长过程中最重要的教育场所，帮助我们学习如何与他人相处、如何做出正确的决定、如何面对挫折和困难}
{"object":"chat.completion","model":"RWKV-toolformer-translation-japanese-chinese-english-7B-World-20230815-ctx128k","usage":{"prompt_tokens":710,"completion_tokens":19,"total_tokens":729},"choices":[{"message":{"role":"assistant","content":" \"name\":\"电影推荐\",\"type\":\"movies\",\"content\":null}"},"index":0,"finish_reason":"stop"}]}
{"name":"电影推荐","type":"movies","content":null}}
{"object":"chat.completion","model":"R

,question,answer
0,你觉得家庭重要吗?,家庭对于绝大多数人来说都是非常重要的。家庭是温暖的避风港，是我们从小成长、获取支持和关爱的地...
1,你喜欢看电影吗?,虽然我无法真正感受电影，但我可以对电影进行分析和推荐。你有什么类型的电影推荐吗？
2,你觉得人们为什么容易陷入负面情绪?,人们容易陷入负面情绪主要是因为对事物的看法和态度产生了偏差。消极的想法和情绪会影响我们的心情...
3,您在医美专栏预约服务的客户编号?,AestheticCustID[MASK]
4,你了解概率论与数理统计的一些重要定理和公式吗?,[MASK]有许多重要的定理和公式，如大数定律、中心极限定理、贝叶斯定理、正态分布等。这些定...
...,...,...
801,你有什么特长技能吗?,作为[MASK]，我的特长是处理和分析大量数据，以及快速学习和适应新知识。
802,您在智能硬件生态参与活动的编号?,[MASK]
803,如何在服装行业赚取一个亿?,在服装行业赚取一个亿需要具备时尚敏锐度和品牌运营能力。首先，要了解消费者的时尚需求和流行趋势...
804,您办理过的健康险/意外险保单号?,[MASK]


In [ ]:
df1.to_csv("merge_shuffle_mask.csv")